In [40]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]

In [58]:
product = "Paprika"
products = inventory.columns[2:]
inventory.columns[-2]

'date'

In [3]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1)

In [4]:
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)

In [5]:
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
promotions_filter = promotions[promotions["description"] == product]

def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0

filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)

<ipython-input-5-bc6caeb2001c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


In [8]:
def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total

#filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

In [9]:
filter_inventory.head()

,day,before or after delivery,date,week,Paprika,discount
0,0,before,2018-01-01,1,[],0
1,0,after,2018-01-01,1,"[(14, 34)]",0
2,3,before,2018-01-04,1,[],0
3,3,after,2018-01-04,1,"[(17, 34)]",0
4,7,before,2018-01-08,2,"[(17, 1)]",0


In [10]:
re_date = []
for element in filter_inventory[filter_inventory['before or after delivery'] == 'before']['day']:
    re_date.append(element)
re_dates = np.unique(re_date)

In [11]:
ship_date = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        ship_date.append(batch[0])
sh_dates = np.unique(ship_date)

In [16]:
len(sh_dates)

16

In [13]:
df_ship = pd.DataFrame({"receivedDate":re_dates, "bestby":sh_dates})
df_ship

ValueError: arrays must all be same length

In [12]:
inventory.head()

,day,before or after delivery,Komkommer,Courgette,SnoepVegetable tomaat,Paprika Mix,Paprika,Geschrapte worteltjes,Basis Sperziebonen,Basis Rode kool met appel,...,Vanille muffins (vers),Basic chocolate chip cookies,Oreo,Bakkersspeculaas,Eierkoeken,Ribbelchips naturel,Ribbelchips paprika,Doritos sweet chili,date,week
0,0,before,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],2018-01-01,1
1,0,after,"[(14, 36)]","[(14, 8)]","[(14, 37)]","[(14, 33)]","[(14, 34)]","[(14, 35)]","[(14, 31)]","[(14, 35)]",...,"[(100, 130)]","[(100, 45)]","[(100, 131)]","[(100, 45)]","[(100, 125)]","[(100, 45)]","[(100, 44)]","[(100, 128)]",2018-01-01,1
2,3,before,"[(14, 13)]",[],"[(14, 13)]","[(14, 11)]",[],[],"[(14, 2)]","[(14, 12)]",...,"[(100, 3)]",[],"[(100, 19)]","[(100, 7)]","[(100, 8)]","[(100, 10)]","[(100, 11)]","[(100, 26)]",2018-01-04,1
3,3,after,"[(14, 13), (17, 36)]","[(17, 8)]","[(14, 13), (17, 37)]","[(14, 11), (17, 33)]","[(17, 34)]","[(17, 35)]","[(14, 2), (17, 31)]","[(14, 12), (17, 35)]",...,"[(100, 3), (103, 130)]","[(103, 45)]","[(100, 19), (103, 131)]","[(100, 7), (103, 45)]","[(100, 8), (103, 125)]","[(100, 10), (103, 45)]","[(100, 11), (103, 44)]","[(100, 26), (103, 128)]",2018-01-04,1
4,7,before,"[(17, 35)]",[],"[(17, 27)]","[(17, 17)]","[(17, 1)]","[(17, 13)]","[(17, 8)]","[(17, 32)]",...,"[(103, 26)]","[(103, 19)]","[(103, 55)]","[(103, 16)]","[(103, 39)]","[(103, 28)]","[(103, 19)]","[(103, 58)]",2018-01-08,2


In [18]:
def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["bestby"]:
                    return batch[1]
#df_ship["amount"] = df_ship.apply(input_inv, axis = 1)

In [21]:
# df_ship
#product gets thrown away on the day it goes bad, no one can buy it anymore ON THAT DAY
#discard any items that has best before for enxt year
#haven't sold anything that is not in stock
# df_ship = df_ship[df_ship["bestby"] < 365]
# df_ship

In [22]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(int(row["bestby"]))
    #print(type(row['best before']))
    return converted_date
def date_converter_inv(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(int(row["day"]))
    #print(type(row['best before']))
    return converted_date
def date_converter_re(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(int(row["receivedDate"]))
    #print(type(row['best before']))
    return converted_date
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

# df_ship["date"] = df_ship.apply(date_converter, axis = 1)
# df_ship["week"] = df_ship.apply(week_of_year, axis = 1)
# df_ship['product'] = [product] * len(df_ship)
# df_week_ship = df_ship.groupby('week').sum()


In [63]:
np.unique(df_ship['amount'])

array([ 7, 14, 34])

In [19]:
df_week_ship.head()

,receivedDate,bestby,amount
week,,,
1,350,364,34
3,3,31,68
4,17,45,68
5,31,59,28
6,45,73,68


In [60]:
agg_df = pd.DataFrame(columns=['week', 'amount', 'product'])

for product in inventory.columns[2:-2]:
    #product = 'Doritos sweet chili'
    inventory["date"] = inventory.apply(date_converter_inv, axis = 1)
    inventory["week"] = inventory.apply(week_of_year, axis = 1)
    filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
    #filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)
    #filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

    re_date = []
    for element in filter_inventory[filter_inventory['before or after delivery'] == 'after']['day']:
        re_date.append(element)
    re_dates = np.unique(re_date)

    ship_date = []
    for element_list in filter_inventory[product]:
        #ship_date.append(element_list[-1][])
        if len(element_list) > 0:
            for batch in element_list:
                #print(type(batch))
                ship_date.append(batch[0])
    sh_dates = np.unique(ship_date)

    print(len(re_dates), len(sh_dates))
    #print(filter_inventory[product])

    df_ship = pd.DataFrame({"receivedDate":re_dates, "bestby":sh_dates})
    df_ship = df_ship[df_ship["bestby"] < 365]
    df_ship["date"] = df_ship.apply(date_converter_re, axis = 1)
    df_ship["week"] = df_ship.apply(week_of_year, axis = 1)
    df_ship["amount"] = df_ship.apply(input_inv, axis = 1)
    df_ship['product'] = [product] * len(df_ship)
    df_week_ship = df_ship.groupby('week').sum().reset_index()
    df_week_ship['product'] = [product] * len(df_week_ship)
    #df_week_ship
    agg_df = agg_df.append(df_week_ship[['week', 'amount', 'product']], ignore_index=True)
    print("Completed {}".format(product))
#df_week_ship



104 104
Completed Komkommer
104 104
Completed Courgette
104 104
Completed SnoepVegetable tomaat
104 104
Completed Paprika Mix
104 104
Completed Paprika
104 104
Completed Geschrapte worteltjes
104 104
Completed Basis Sperziebonen
104 104
Completed Basis Rode kool met appel
104 104
Completed Basis worteltjes fijn
104 104
Completed Basis erwten zeer fijn
104 104
Completed Basis sperziebonen
104 104
Completed Biologische courgette
104 104
Completed Biologische Bloemkool
104 104
Completed Biologische knoflook
104 104
Completed Biologische pompoen
104 104
Completed Biologische kikkererwten
104 104
Completed Biologische prei
104 104
Completed Biologische rode kool
104 104
Completed Biologische cherry tomaten
104 104
Completed Biologische brocolli
104 104
Completed Boerenkool gesneden
104 104
Completed Spruiten
104 104
Completed Witlof
104 104
Completed Rucola
104 104
Completed Nasi bami Vegetable
104 104
Completed Snijbonen
104 104
Completed Buitenbeentjes puntpaprika
104 104
Completed Basis 

In [104]:
df_agg = agg_df
df_agg['amount'] = df_agg['amount'].astype(int)
m_std_df = df_agg.groupby('product').agg({'amount': ['mean', 'std']})
m_std_df

amount           
                                                  mean        std
product                                                          
Aardbeien                                    48.720000   9.014818
Aardbeien ontkroond                          55.480000   4.550129
Amandelkrans                                 39.307692   4.992302
Atkins laag koolhydraat volkoren brood half  41.442308   6.300991
Bakkersspeculaas                             86.631579  16.000711
...                                                ...        ...
gerookt onbijtspek                           31.490196   7.734009
half of half gehakt                          33.215686   4.100311
meergranen volkoren half                     36.250000   6.522585
rundergehakt                                 31.882353   4.407480
rundervinken                                 31.764706   3.972849

[155 rows x 2 columns]

In [136]:
filt_cat = m_std_df.index[:2]
filt_cat = m_std_df[('amount', 'std')].sort_values(ascending = False).index[0:5]
df_agg_filt = df_agg[df_agg['product'].isin(filt_cat)]

In [145]:
import plotly.express as px
fig = px.box(df_agg_filt, x="product", y="amount", color='product', title='Highest std dev Purchases from supplier', labels={'amount': "Quantity per week", "product":"Product"})
fig.update_layout(showlegend=False)
fig.show()

In [107]:
m_std_df[('amount', 'mean')].sort_values(ascending = False)

product
Basis annanasstukjes                283.280000
Oreo                                264.736842
Eierkoeken                          257.894737
Doritos sweet chili                 254.578947
Vanille muffins (vers)              243.578947
                                       ...    
Biologische rundersoepballetjes      11.686275
Biologische runderchipolata          10.156863
Biologische spekreepjes              10.000000
biologische Hamburger                 9.882353
Biologische gerookte ontbijtspek      9.843137
Name: (amount, mean), Length: 155, dtype: float64

In [160]:
df_std = pd.DataFrame(m_std_df[('amount', 'std')].sort_values(ascending = False))[:30]
df_std.reset_index(level=0)
df_std[('amount', 'std')]

product
Vanille muffins (vers)                  50.001422
Basis annanasstukjes                    44.005120
Doritos sweet chili                     32.568955
Eierkoeken                              27.327631
Blauwe bessen.1                         22.148331
Blauwe bessen                           22.148331
Basic chocolate chip cookies            18.197353
Oreo                                    16.871028
Ribbelchips paprika                     16.033127
Bakkersspeculaas                        16.000711
Rundergehakt.1                          15.445388
Rundergehakt                            15.445388
Basis Rode kool met appel               15.269680
Basic casino half                       14.847376
Basic meergranen heel                   14.682307
Basic wit heel                          14.585616
Basic volkoren heel                     14.422414
Biologisch volkoren heel                14.287450
Courgette                               14.014222
Basis Maiskorrels                       12

In [173]:

fig = px.bar(x=df_std.index, y=df_std[('amount', 'std')], color = df_std.index, title = "Descending products by std dev weekly restock", labels = {"y":"Weekly Restock Std Dev","x":"Product"})
fig.update_layout(showlegend=False)
fig.update_xaxes(tickangle = 45)
fig.show()

In [85]:
inventory[['day', 'before or after delivery', 'Doritos sweet chili']]

,day,before or after delivery,Doritos sweet chili
0,0,before,[]
1,0,after,"[(100, 128)]"
2,3,before,"[(100, 26)]"
3,3,after,"[(100, 26), (103, 128)]"
4,7,before,"[(103, 58)]"
...,...,...,...
203,353,after,"[(401, 100), (404, 128), (408, 128), (411, 128..."
204,357,before,"[(404, 120), (408, 128), (411, 128), (415, 128..."
205,357,after,"[(404, 120), (408, 128), (411, 128), (415, 128..."
206,360,before,"[(404, 35), (408, 128), (411, 128), (415, 128)..."


In [61]:
agg_df['product'].unique()

array(['Komkommer', 'Courgette', 'SnoepVegetable tomaat', 'Paprika Mix',
       'Paprika', 'Geschrapte worteltjes', 'Basis Sperziebonen',
       'Basis Rode kool met appel', 'Basis worteltjes fijn',
       'Basis erwten zeer fijn', 'Basis sperziebonen',
       'Biologische courgette', 'Biologische Bloemkool',
       'Biologische knoflook', 'Biologische pompoen',
       'Biologische kikkererwten', 'Biologische prei',
       'Biologische rode kool', 'Biologische cherry tomaten',
       'Biologische brocolli', 'Boerenkool gesneden', 'Spruiten',
       'Witlof', 'Rucola', 'Nasi bami Vegetable', 'Snijbonen',
       'Buitenbeentjes puntpaprika', 'Basis Maiskorrels', 'Wortelen',
       'Pompoen stukjes', 'Verse zuurkool', 'Buitenbeentjes komkommer',
       'Buitenbeentjes paprika', 'SnoepVegetable worteltjes',
       'WokVegetable thais', 'Mandarijenen', 'Bananen',
       'Chiquita bananen', 'Pitloze witte druiven',
       'Zoete kleine appeltjes', 'Pink lady appels',
       'Pitloze rode dru

In [147]:
agg_df[agg_df['product'] == 'Basic casino half']

,week,amount,product
6884,1,70,Basic casino half
6885,2,70,Basic casino half
6886,3,70,Basic casino half
6887,4,70,Basic casino half
6888,5,70,Basic casino half
6889,6,28,Basic casino half
6890,7,70,Basic casino half
6891,8,70,Basic casino half
6892,9,70,Basic casino half
6893,10,70,Basic casino half
